In [1]:
import datetime
from cryptography.hazmat.primitives import serialization
import requests
import pandas as pd

In [2]:
pd.options.display.max_columns = 100

In [3]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [4]:
HTTPResponse = requests.packages.urllib3.response.HTTPResponse
orig_HTTPResponse__init__ = HTTPResponse.__init__
def new_HTTPResponse__init__(self, *args, **kwargs):
    orig_HTTPResponse__init__(self, *args, **kwargs)
    try:
        self.peer_certificate = self._connection.peer_certificate
    except AttributeError:
        pass
HTTPResponse.__init__ = new_HTTPResponse__init__

HTTPAdapter = requests.adapters.HTTPAdapter
orig_HTTPAdapter_build_response = HTTPAdapter.build_response
def new_HTTPAdapter_build_response(self, request, resp):
    response = orig_HTTPAdapter_build_response(self, request, resp)
    try:
        response.peer_certificate = resp.peer_certificate
    except AttributeError:
        pass
    return response
HTTPAdapter.build_response = new_HTTPAdapter_build_response

HTTPSConnection = requests.packages.urllib3.connection.HTTPSConnection
orig_HTTPSConnection_connect = HTTPSConnection.connect
def new_HTTPSConnection_connect(self):
    orig_HTTPSConnection_connect(self)
    try:
        self.peer_certificate = self.sock.connection.get_peer_certificate()
    except AttributeError:
        pass
HTTPSConnection.connect = new_HTTPSConnection_connect

In [5]:
def check_certificate(target, method=requests.head, verify=False, timeout=20.):
    tic = datetime.datetime.utcnow()
    response = method(target, timeout=timeout, verify=verify)
    tac = datetime.datetime.utcnow()
    print(response, target)
    certificate = response.peer_certificate
    print(certificate)
    result = {
        'request': {
            'target': target,
            'method': method.__name__.upper(),
            'status': response.status_code,
            'timestamp': tic,
            'elapsed': (tac - tic)/datetime.timedelta(seconds=1),
            'verify': verify,
            'timeout': timeout,            
            #'headers': dict(response.headers)
        },
        'certificate': {
            'subject': {k.decode(): v.decode() for k, v in dict(certificate.get_subject().get_components()).items()},
            'issuer': {k.decode(): v.decode() for k, v in dict(certificate.get_issuer().get_components()).items()},
            'serial': certificate.get_serial_number().to_bytes(20, byteorder='big').hex(),
            'version': certificate.get_version(),
            'algorithm': certificate.get_signature_algorithm().decode(),
            'key_size': certificate.get_pubkey().bits(),
            'key_type': certificate.get_pubkey().type(),
            'digest': certificate.digest("sha1").decode(),
            'start': datetime.datetime.strptime(certificate.get_notBefore().decode(), '%Y%m%d%H%M%SZ'),
            'stop': datetime.datetime.strptime(certificate.get_notAfter().decode(), '%Y%m%d%H%M%SZ'),
            'expired': certificate.has_expired(),
            'pem': certificate.to_cryptography().public_bytes(encoding=serialization.Encoding.PEM).decode()
        }
    }
    result["certificate"]["expires"] = (result["certificate"]["stop"] - result["request"]["timestamp"])/datetime.timedelta(days=1)
    return result

In [6]:
targets = [
    "https://ulb.be",
    "https://auth.ulb.be",
    "https://bib.ulb.be",
    "https://bib-ulb-be.ezproxy.ulb.ac.be",
    "https://sso-cas.ulb.ac.be",
    "https://cibleplus.ulb.ac.be",
    "https://gehol.ulb.ac.be",
    "https://www.pub-ulb.be",
    "https://cirem.ulb.be",
]

In [7]:
results = []
for target in targets:
    result = check_certificate(target)
    results.append(result)

<Response [301]> https://ulb.be
<Response [302]> https://auth.ulb.be
<Response [200]> https://bib.ulb.be
<Response [302]> https://bib-ulb-be.ezproxy.ulb.ac.be
<Response [404]> https://sso-cas.ulb.ac.be
<Response [302]> https://cibleplus.ulb.ac.be
<Response [200]> https://gehol.ulb.ac.be
<Response [200]> https://www.pub-ulb.be
<Response [200]> https://cirem.ulb.be


In [8]:
df = pd.json_normalize(results)
df.columns = [column.replace('.', '_').lower() for column in df.columns]
df

,request_target,request_method,request_status,request_timestamp,request_elapsed,request_verify,request_timeout,certificate_subject_c,certificate_subject_st,certificate_subject_l,certificate_subject_o,certificate_subject_ou,certificate_subject_cn,certificate_issuer_c,certificate_issuer_st,certificate_issuer_l,certificate_issuer_o,certificate_issuer_cn,certificate_serial,certificate_version,certificate_algorithm,certificate_key_size,certificate_key_type,certificate_digest,certificate_start,certificate_stop,certificate_expired,certificate_pem,certificate_expires,certificate_subject_postalcode,certificate_subject_street
0,https://ulb.be,HEAD,301,2021-05-17 08:41:43.497545,0.099562,False,20.0,BE,Brussels,Brussels,Université libre de Bruxelles,Cellule Web,www.ulb.be,NL,Noord-Holland,Amsterdam,TERENA,TERENA SSL CA 3,0000000009a097f642a2e69b00c72c782371f777,2,sha256WithRSAEncryption,2048,6,8A:92:E5:49:15:9C:5F:C1:4F:D1:3B:85:D8:0F:57:0...,2020-03-13,2022-03-18 12:00:00,False,-----BEGIN CERTIFICATE-----\nMIILXDCCCkSgAwIBA...,305.137691,NaN,NaN
1,https://auth.ulb.be,HEAD,302,2021-05-17 08:41:43.601169,0.247076,False,20.0,BE,Bruxelles-Capitale,Bruxelles,Université Libre de Bruxelles,IT,auth.ulb.be,NL,NaN,NaN,GEANT Vereniging,GEANT OV RSA CA 4,00000000d3bcb7eeedcccec5d1510e019ca04e10,2,sha384WithRSAEncryption,2048,6,12:65:C2:6F:04:29:6B:D2:FC:08:8D:65:C4:8B:74:7...,2020-07-02,2021-07-02 23:59:59,False,-----BEGIN CERTIFICATE-----\nMIIHjTCCBXWgAwIBA...,46.637678,1050,Avenue Franklin Roosevelt 50
2,https://bib.ulb.be,HEAD,200,2021-05-17 08:41:43.850636,0.669111,False,20.0,BE,Brussels,Brussels,Université libre de Bruxelles,Cellule Web,www.ulb.be,NL,Noord-Holland,Amsterdam,TERENA,TERENA SSL CA 3,0000000009a097f642a2e69b00c72c782371f777,2,sha256WithRSAEncryption,2048,6,8A:92:E5:49:15:9C:5F:C1:4F:D1:3B:85:D8:0F:57:0...,2020-03-13,2022-03-18 12:00:00,False,-----BEGIN CERTIFICATE-----\nMIILXDCCCkSgAwIBA...,305.137687,NaN,NaN
3,https://bib-ulb-be.ezproxy.ulb.ac.be,HEAD,302,2021-05-17 08:41:44.521633,0.457917,False,20.0,BE,Bruxelles-Capitale,Bruxelles,Université Libre de Bruxelles,SISC,*.ezproxy.ulb.ac.be,NL,NaN,NaN,GEANT Vereniging,GEANT OV RSA CA 4,00000000ec00cf3e1499f357873896ab9b038c36,2,sha384WithRSAEncryption,2048,6,AE:AB:17:1B:CB:DD:04:6D:D4:81:B2:04:2B:60:A3:4...,2020-07-16,2022-07-16 23:59:59,False,-----BEGIN CERTIFICATE-----\nMIIH1TCCBb2gAwIBA...,425.637668,1050,Avenue Franklin Roosevelt 50
4,https://sso-cas.ulb.ac.be,HEAD,404,2021-05-17 08:41:44.981229,0.092134,False,20.0,BE,Bruxelles-Capitale,Bruxelles,Université Libre de Bruxelles,Centre de Calcul,sso-cas.ulb.ac.be,NL,NaN,NaN,GEANT Vereniging,GEANT OV RSA CA 4,000000002cba4056867a14d8f520e9d6e5247a92,2,sha384WithRSAEncryption,2048,6,65:A8:94:BF:DE:58:70:48:B1:46:A2:97:64:68:CF:E...,2020-06-07,2022-06-07 23:59:59,False,-----BEGIN CERTIFICATE-----\nMIIHqTCCBZGgAwIBA...,386.637662,NaN,NaN
5,https://cibleplus.ulb.ac.be,HEAD,302,2021-05-17 08:41:45.074903,0.396166,False,20.0,BE,"Bruxelles-Capitale, Région de",Bruxelles,Université Libre de Bruxelles,NaN,cibleplus.ulb.ac.be,NL,NaN,NaN,GEANT Vereniging,GEANT OV RSA CA 4,000000007ffc6c1ef99aa9242b9f0e3898f0b744,2,sha384WithRSAEncryption,2048,6,87:B2:2A:D9:1A:43:DF:25:59:00:A1:60:D1:9A:CB:B...,2020-12-17,2021-12-17 23:59:59,False,-----BEGIN CERTIFICATE-----\nMIIHRjCCBS6gAwIBA...,214.637661,1050,Avenue Franklin Roosevelt 50
6,https://gehol.ulb.ac.be,HEAD,200,2021-05-17 08:41:45.472615,0.316536,False,20.0,BE,NaN,Bruxelles,Université libre de Bruxelles,IT,gehol.ulb.ac.be,NL,Noord-Holland,Amsterdam,TERENA,TERENA SSL CA 3,00000000061566d7a9e1bdafcdc158abbce10f39,2,sha256WithRSAEncryption,2048,6,2E:31:33:65:31:B8:0D:D0:92:9A:6D:E5:70:AF:51:6...,2019-09-11,2021-09-15 12:00:00,False,-----BEGIN CERTIFICATE-----\nMIIG0jCCBbqgAwIBA...,121.137668,NaN,NaN
7,https://www.pub-ulb.be,HEAD,200,2021-05-17 08:41:45.791154,0.158918,False,20.0,NaN,NaN,NaN,NaN,NaN,www.pub-ulb.be,FR,Paris,Paris,Gandi,Gandi Standard SSL CA 2,00000000c0c8e35e562437efe2075719ce30ceb5,2,sh